In [1]:
import pandas as pd
import sys
sys.path.append("../code/data")
import utils
import json
import string
import re
import numpy as np
import collections
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
df_title_akas = pd.read_csv("../data/imdb/title.akas.tsv",sep='\t',header='infer')

/Users/mihirkale.s/PycharmProjects/entity/venv/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df_title_basics = pd.read_csv("../data/imdb/title.basics.tsv",sep='\t',header='infer')

/Users/mihirkale.s/PycharmProjects/entity/venv/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
translator = str.maketrans('', '', string.punctuation)
df_title_basics['primaryTitle'] = df_title_basics['primaryTitle'].str.lower()
df_title_basics['primaryTitle'] = df_title_basics['primaryTitle'].apply(lambda x : str(x).translate(translator))

In [5]:
df_title_basics['genres'] = df_title_basics['genres'].str.lower()

In [6]:
df_title_akas['title'] = df_title_akas['title'].str.lower()
df_title_akas['title'] = df_title_akas['title'].apply(lambda x : str(x).translate(translator))

In [7]:
#df_title_basics[df_title_basics['genres'] == 'Romance']

In [8]:
traindata = [json.loads(line) for line in open("../data/mit-movie-1/original/engtrain.json")]
devdata = [json.loads(line) for line in open("../data/mit-movie-1/original/engdev.json")]

In [9]:
def get_titles(data):
    titles = []
    for text,annotations in data:
        if 'title' not in annotations:continue
        title_list = annotations.get('title')
        for title in title_list: titles.append(title[0])
    return titles

def get_title_counts(titles):
    titles_count = collections.defaultdict(int)
    for title in titles : titles_count[title] += 1
    return titles_count

In [22]:
#df_title_akas.join(df_title_basics,on='titleId')

In [43]:
df_titles = pd.merge(df_title_basics,df_title_akas, how = 'left', right_on = 'titleId', left_on = 'tconst')

In [53]:
df_titles = df_titles[ (df_titles['titleType'] == 'movie') ]

In [54]:
df_titles['is_romance'] = df_titles['genres'].apply(lambda x : 'romance' in str(x).split(","))
df_titles['is_action'] = df_titles['genres'].apply(lambda x : 'action' in str(x).split(","))

/Users/mihirkale.s/PycharmProjects/entity/venv/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/Users/mihirkale.s/PycharmProjects/entity/venv/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [55]:
df_titles['startYearInt'] = df_titles['startYear'].apply(lambda x : int(x) if x!="\\N" else -1)

/Users/mihirkale.s/PycharmProjects/entity/venv/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [56]:
hindi_titles = list(set(df_titles[  (df_titles['region'] == 'IN') ]['primaryTitle'].values))
us_titles = list(set(df_titles[  (df_titles['region'] == 'US') ]['primaryTitle'].values))

In [57]:
recent_titles = list(set(df_titles[  (df_titles['region'] == 'US') & (df_titles['startYearInt'] > 2010)]['primaryTitle'].values))
recent_titles = [title for title in recent_titles if len(title) > 0 ]

In [58]:
old_titles = list(set(df_titles[  (df_titles['region'] == 'US') & (df_titles['startYearInt'] < 1980) & (df_titles['startYearInt'] > 1950)]['primaryTitle'].values))
old_titles = [title for title in old_titles if len(title) > 0 ]

In [61]:
#df_titles[  (df_titles['region'] == 'IN') ]

In [62]:
from collections import defaultdict
title2genre = dict(df_titles[  (df_titles['region'] == 'US') ][['primaryTitle','genres']].values)
genre2title = defaultdict(set)
for title,genre_str in title2genre.items():
    genres = genre_str.split(",")
    for genre in genres : genre2title[genre].add(title)
#df_titles[df_titles['is_action'] == True]

In [63]:
def write_to_bio_file(f,tokens,tags):
    for token,tag in zip(tokens,tags):
        f.write(tag + "\t" + token + "\n")
    f.write("\n")
    

In [64]:


def create_dataset(entities,slot,in_path,in_ann_path,out_path):
    sentence_g = utils.sentence_generator(in_path)
    tagged_sentences = [k for k in sentence_g]
    dataset = [json.loads(line) for line in open(in_ann_path)]
    f = open(out_path,"w")    
    
    for sentence_idx,tagged_sentence in enumerate(tagged_sentences):

        text,annotations = dataset[sentence_idx]
        tokens = [token for (token, _) in tagged_sentence]
        tags = [tag for (_, tag) in tagged_sentence]
        
        if slot not in annotations : 
            write_to_bio_file(f,tokens,tags)
            continue
            
        _,start_idx,end_idx = annotations[slot][0]
        tokens = text.split()
        random_entity_tokens = np.random.choice(entities).split()
        random_entity_tags = ['I-'+slot.upper() for token in random_entity_tokens]
        random_entity_tags[0] = 'B-' + slot.upper()

        lhs_tokens,lhs_tags = tokens[:start_idx],tags[:start_idx]
        rhs_tokens,rhs_tags = tokens[end_idx+1:],tags[end_idx+1:]

        new_tokens = lhs_tokens + random_entity_tokens + rhs_tokens
        new_tags = lhs_tags + random_entity_tags + rhs_tags

        write_to_bio_file(f,new_tokens,new_tags)
    f.close()

In [71]:
dev_inn_ann_path = "../data/mit-movie-1/original/engdev.json"
dev_in_path = "../data/mit-movie-1/original/engdev.bio"

train_inn_ann_path = "../data/mit-movie-1/original/engtrain.json"
train_in_path = "../data/mit-movie-1/original/engtrain.bio"

romance_titles = np.array(list(genre2title['romance']))
action_titles = np.array(list(genre2title['action']))

out_path = "../data/mit-movie-1/romance_dev.bio"
create_dataset(romance_titles,'title',dev_in_path,dev_inn_ann_path,out_path)
out_path = "../data/mit-movie-1/romance_train.bio"
create_dataset(romance_titles,'title',train_in_path,train_inn_ann_path,out_path) 

out_path = "../data/mit-movie-1/action_dev.bio"
create_dataset(action_titles,'title',dev_in_path,dev_inn_ann_path,out_path)
out_path = "../data/mit-movie-1/action_train.bio"
create_dataset(action_titles,'title',train_in_path,train_inn_ann_path,out_path) 

out_path = "../data/mit-movie-1/old_dev.bio"
create_dataset(old_titles,'title',dev_in_path,dev_inn_ann_path,out_path)
out_path = "../data/mit-movie-1/old_train.bio"
create_dataset(old_titles,'title',train_in_path,train_inn_ann_path,out_path) 

out_path = "../data/mit-movie-1/recent_dev.bio"
create_dataset(recent_titles,'title',dev_in_path,dev_inn_ann_path,out_path)
out_path = "../data/mit-movie-1/recent_train.bio"
create_dataset(recent_titles,'title',train_in_path,train_inn_ann_path,out_path) 

In [72]:
out_path = "../data/mit-movie-1/bollywood_dev.bio"
create_dataset(hindi_titles,'title',dev_in_path,dev_inn_ann_path,out_path)
out_path = "../data/mit-movie-1/bollywood_train.bio"
create_dataset(hindi_titles,'title',train_in_path,train_inn_ann_path,out_path) 

out_path = "../data/mit-movie-1/us_dev.bio"
create_dataset(us_titles,'title',dev_in_path,dev_inn_ann_path,out_path)
out_path = "../data/mit-movie-1/us_train.bio"
create_dataset(us_titles,'title',train_in_path,train_inn_ann_path,out_path) 